In [1]:
import os
import sys

# certify the notebook is able to find "deck_class" folder
current_directory = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_directory, '..')))

In [2]:
from deck_class.truco import Deck_of_Truco
from dataclasses import dataclass
import numpy as np

# Game modeling

Similar to `struct` in C, Python has the native `dataclasses` module that implements the `@dataclass` decorator for automatically adding generated special methods such as `__init__()`.

**Player** attributes:
1. `hand` -> list with three cards
2. `index` -> int value for reference (0 or 1)


**Team** attributes:
1. `players` -> list with two instances of **Player**
2. `index` -> int value for reference (0 or 1)

For ingame reference, each player will be indentified by the tuple `(Team.index, Player.index)`.

In [20]:
@dataclass
class Player:
    hand: list
    index: int
    
    
@dataclass
class Team:
    players: list
    index: int

# Class `Truco_Game`

This class will consist of handling the game state, certifying that the rules of the game are followed and providing methods that enable the agents to play the game.

* Definition of state ($s$):
1. $P_s$ = player that can take action at the current state $s$
2. $C_P$ = player and teammate's cards 
3. $p_0$ = points earned by team 0
4. $p_1$ = points earned by team 1
5. $p_s$ = points at stake in the current round
6. $T_C$ = current cards on the table
7. $F_P$ = first player of current round (mão)
8. $B$ = if the previous player raised the round value
9. $W$ = array with the team index which won $i$-th round ($W = [w_0, w_1, w_2]$)
10. $n_r$ = number of round current beeing played

$$s = (P_s, C_P, p_0, p_1, p_s, T_C, F_P, B, W, n_r)$$

Obs.: if the class was instantiated with 4 players $C_P$ will hide teammates cards. 

In [71]:
class Truco_Game():
    
    def __init__(self, max_points=12):
        self._deck = Deck_of_Truco()        
        self._goal = max_points
        self._teams = self.draw_teams_hands()
        self._first_to_play = (0, 0)
        self._last_to_play = (1, 1)
        self._screamer = None
        
        
    def draw_teams_hands(self):
        player_11 = Player(self._deck.draw_hand(), 0)
        player_12 = Player(self._deck.draw_hand(), 0)
        player_21 = Player(self._deck.draw_hand(), 1)
        player_22 = Player(self._deck.draw_hand(), 1)
        
        team_1 = Team([player_11, player_21], 0)
        team_2 = Team([player_12, player_22], 1)
        
        return [team_1, team_2]
    
        
    def start_state(self):
        team_index = 0
        player_index = 0
        
        first_player = (team_index, player_index)
        player_hand = self.get_player_hand(first_player)
        
        points_t1, points_t2 = 0, 0
        round_value = 1
        raise_call = 0
        
        return (first_player, player_hand, points_t1, points_t2, round_value, raise_call)
    
    
    
    def get_player_hand(self, player):
        team_index = player[0]
        player_index = player[1]
        
        return self._teams[team_index].players[player_index].hand
    
    
    def successor(self, state, action):
        player, hand, points_t1, points_t2, round_value, raise_call = state
        current_team, current_player = player
        
        
        if action in hand:
            card_played = action
            
            self.remove_card_from_hand(player, card_played)
            next_player = self.forward_player(player)
            
        elif action == 'run':
            if current_team == 0:
                points_t2 += round_value
            else:
                points_t1 += round_value
                
            raise_call, round_value = 0, 1
            
            self.start_new_round()
            next_player = self._first_to_play
            
        elif action == 'raise':
            if raise_call:
                if round_value == 1:
                    round_value = 3
                else:
                    round_value += 3
            else:
                self._screamer = player
             
            next_player = self.decide_next_bettor(player, round_value)
            raise_call = 1
        
        elif action == 'go':
            raise_call = 0
            
            next_player = self._screamer
            self._screamer = None
            
            if round_value == 1:
                round_value = 3
            else:
                round_value += 3
       
    
        next_hand = self.get_player_hand(next_player)
        
        return (next_player, next_hand, points_t1, points_t2, round_value, raise_call)
    
    
    def start_new_round(self):
        self._teams = self.draw_teams_hands()
        self._first_to_play = self.forward_player(self._first_to_play)
        self._last_to_play = self.forward_player(self._last_to_play)
        return
    
    
    def remove_card_from_hand(self, player, card):
        hand = self._teams[player[0]].players[player[1]].hand
        self._teams[player[0]].players[player[1]].hand = np.delete(hand, np.where(hand == card))
        return
    
    
    def decide_next_bettor(self, player, round_value):
        if (round_value % 2 == 0) or round_value == 1:
            new_player = self.forward_player(player)
        else:
            new_player = self.backward_player(player)
            
        return new_player
             
    
    def forward_player(self, player):
        team_index = player[0]
        player_index = player[1]
        
        if team_index == 1:
            new_player_index = (player_index + 1)%2
        else:
            new_player_index = player_index
        
        new_team_index = (team_index + 1)%2
        
        return (new_team_index, new_player_index)
    
    
    def backward_player(self, player):
        team_index = player[0]
        player_index = player[1]
        
        if team_index == 1:
            new_player_index = player_index
        else:
            new_player_index = (player_index + 1)%2
            
        new_team_index = (team_index + 1)%2
        
        return (new_team_index, new_player_index)
    
    
    def possible_actions(self, state):
        player, hand, points_t1, points_t2, round_value, raise_call = state
         
        if raise_call:
            possible_actions = ['go', 'run']
        else:
            possible_actions = list(self.get_player_hand(player))
            
        if round_value <= 9 and max(points_t1, points_t2) < 11:
            possible_actions.append('raise')
        
        return possible_actions
        
    
    def player(self, state):
        player, hand, points_t1, points_t2, round_value, raise_call = state
        return player
      
    def is_end(start, state):
        points_t1, points_t2 = state[2], state[3]
        return points_t1 >= 12 or points_t2 >= 12

In [72]:
def humanPolicy(game, state):
    while True:
        actions = game.possible_actions(state)
        action = input(f'P({state[0]})\n-> actions: {actions}: ')
        
        if action in actions:
            return action

def run_game():
    game = Truco_Game()
    state = game.start_state()
    
    while not game.is_end(state):
        print("Current state ", '='*10, state)
        
        player = game.player(state)
        action = humanPolicy(game, state)
        state = game.successor(state, action)

In [73]:
run_game()

In [74]:
game.possible_actions(s1)

['D2', 'C4', 'H7', 'raise']

In [75]:
game.successor(s1, 'H7')

((1, 0), array(['SA', 'S3', 'HA'], dtype='<U2'), 0, 0, 1, 0)

In [65]:
game._teams[1].players[1].hand

array(['SA', 'H2'], dtype='<U2')

In [64]:
game._teams

[Team(players=[Player(hand=array(['C3', 'HA', 'HQ'], dtype='<U2'), index=1), Player(hand=array(['SJ', 'SQ', 'DK'], dtype='<U2'), index=-1)], index=1),
 Team(players=[Player(hand=array(['SJ', 'SQ', 'DK'], dtype='<U2'), index=-1), Player(hand=array(['SA', 'H2'], dtype='<U2'), index=-1)], index=-1)]